In [1]:
# https://www.coffeebeankorea.com/robots.txt
# User-agent: *
# Allow: /
# 수집 모두 허용 확인.

# 각 매장의 정보를 가져와서,
# 파일 기반 저장 :csv, json
# 데이터베이스 저장, 마리아 디비

# 커피빈 사이트 -> 매장 찾기
# https://www.coffeebeankorea.com/store/store.asp


In [2]:
# 해당 사이트, 매장 찾기
# 자세히 보기 커서를 가까이 가져가면 -> 자바스크립트 함수 명 확인
# 예) storePop2('33') 이런 형식의 함수를 호출
# 코엑스웨스틴서울점262m -> 142 -> storePop2('142')

# 각 요소를 분석
# 1. 매장명 가져오기.
# div 태그, class="store_txt" -> 하위에 h2 태그로

# 2. 주소 가져오기. 3번째요소 -> 인덱스 :2
# div 태그, class="store_txt" -> 하위에 테이블 태그, class="store_table" ->
# tbody 태그 -> tr 태그 -> td 태그

# 3. 전화번호 가져오기. 4번째요소 -> 인덱스 :3
# div 태그, class="store_txt" -> 하위에 테이블 태그, class="store_table" ->
# tbody 태그 -> tr 태그 -> td 태그

In [3]:
import time
import json
import pandas as pd
import pymysql
from bs4 import BeautifulSoup
from selenium import webdriver

def CoffeeBean_store():
    CoffeeBean_URL = "https://www.coffeebeankorea.com/store/store.asp"
    wd = webdriver.Chrome()

    result = []

    for i in range(1, 10):  # 마지막 매장번호(최신 매장까지 크롤링),1~395
        wd.get(CoffeeBean_URL)
        time.sleep(1)  # 웹페이지 연결 대기

        try:
            wd.execute_script(f"storePop2({i})")
            time.sleep(3)  # 스크립트 실행 대기
            html = wd.page_source
            soupCB = BeautifulSoup(html, 'html.parser')

            # 매장명 가져오기
            # 1. 매장명 가져오기.
            # div 태그, class="store_txt" -> 하위에 h2 태그로
            store_name_h2 = soupCB.select("div.store_txt > h2")
            store_name = store_name_h2[0].string.strip() if store_name_h2 else "정보 없음"

            #유효성 체크 걸기, 즉 매장 이름 -> 정보 없음 건너뛰기
            if store_name == "정보 없음":
                print(f"[{i}] 매장 정보가 존재하지 않아 건너뜁니다.")
                continue

            # 주소 및 전화번호 가져오기
            # 2. 주소 가져오기. 3번째요소 -> 인덱스 :2
            # div 태그, class="store_txt" -> 하위에 테이블 태그, class="store_table" ->
            # tbody 태그 -> tr 태그 -> td 태그

            # 3. 전화번호 가져오기. 4번째요소 -> 인덱스 :3
            # div 태그, class="store_txt" -> 하위에 테이블 태그, class="store_table" ->
            # tbody 태그 -> tr 태그 -> td 태그
            store_info = soupCB.select("div.store_txt > table.store_table > tbody > tr > td")
            store_address = store_info[2].text.strip() if len(store_info) > 2 else "주소 없음"
            store_phone = store_info[3].text.strip() if len(store_info) > 3 else "전화번호 없음"

            print(f"📍 매장명: {store_name} | 📍 주소: {store_address} | 📞 전화번호: {store_phone}")

            result.append({"index": i,"store": store_name, "address": store_address, "phone": store_phone})

        except Exception as e:
            print(f"❌ 오류 발생: {e}")
            continue

    wd.quit()
    return result

In [4]:
# JSON 파일 저장 ---
def save_to_json(data):
    json_file = "CoffeeBean.json"
    with open(json_file, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"✅ JSON 파일 저장 완료: {json_file}")

# MariaDB(MySQL) 저장 ---
def save_to_mariadb(data):
    db = pymysql.connect(
        host="localhost",
        user="webuser",  # 변경 필요
        password="webuser",  # 변경 필요
        database="webdb"
    )
    cursor = db.cursor()

    # 테이블 생성 (처음 실행 시 필요)
    create_table_query = """
    CREATE TABLE IF NOT EXISTS coffeebean_stores (
        id INT AUTO_INCREMENT PRIMARY KEY,
        store VARCHAR(255) NOT NULL,
        address TEXT NOT NULL,
        phone VARCHAR(50)
    )
    """
    cursor.execute(create_table_query)

    # 데이터 저장 쿼리
    insert_query = """
    INSERT INTO coffeebean_stores (store, address, phone)
    VALUES (%s, %s, %s)
    """

    try:
        cursor.executemany(insert_query, [(d["store"], d["address"], d["phone"]) for d in data])
        db.commit()
        print(f"✅ {len(data)}개의 매장 데이터가 MariaDB에 저장되었습니다.")
    except Exception as e:
        db.rollback()
        print(f"❌ 데이터 저장 중 오류 발생: {e}")

    cursor.close()
    db.close()

In [5]:
def main():
    print("🚀 CoffeeBean 매장 정보 크롤링 시작...")
    store_data = CoffeeBean_store()

    print(f"store_data 확인 : {store_data} ")
    # 파일 기반 저장,
    print("\n💾 JSON 파일 저장 중...")
    save_to_json(store_data)
    # 데이터베이스 저장.
    print("\n📂 MariaDB 저장 중...")
    save_to_mariadb(store_data)

    print("\n🎉 모든 작업 완료!")

if __name__ == '__main__':
    main()

🚀 CoffeeBean 매장 정보 크롤링 시작...
[1] 매장 정보가 존재하지 않아 건너뜁니다.
❌ 오류 발생: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=140.0.7339.81); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
	GetHandleVerifier [0x0x7ff74938fca5+79861]
	GetHandleVerifier [0x0x7ff74938fd00+79952]
	(No symbol) [0x0x7ff74910cada]
	(No symbol) [0x0x7ff7490f8995]
	(No symbol) [0x0x7ff74911dbda]
	(No symbol) [0x0x7ff749194346]
	(No symbol) [0x0x7ff7491b48c2]
	(No symbol) [0x0x7ff74918c8b3]
	(No symbol) [0x0x7ff749155272]
	(No symbol) [0x0x7ff749156043]
	GetHandleVerifier [0x0x7ff74964b9dd+2946349]
	GetHandleVerifier [0x0x7ff749645c5a+2922410]
	GetHandleVerifier [0x0x7ff7496659e7+3052855]
	GetHandleVerifier [0x0x7ff7493aaa8e+189918]
	GetHandleVerifier [0x0x7ff7493b2a2f+222591]
	GetHandleVerifier [0x0x7ff749398ac4

InvalidSessionIdException: Message: invalid session id; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
	GetHandleVerifier [0x0x7ff74938fca5+79861]
	GetHandleVerifier [0x0x7ff74938fd00+79952]
	(No symbol) [0x0x7ff74910c8fc]
	(No symbol) [0x0x7ff74915438d]
	(No symbol) [0x0x7ff74918c9a2]
	(No symbol) [0x0x7ff749187374]
	(No symbol) [0x0x7ff74918642a]
	(No symbol) [0x0x7ff7490d7075]
	GetHandleVerifier [0x0x7ff74964b9dd+2946349]
	GetHandleVerifier [0x0x7ff749645c5a+2922410]
	GetHandleVerifier [0x0x7ff7496659e7+3052855]
	GetHandleVerifier [0x0x7ff7493aaa8e+189918]
	GetHandleVerifier [0x0x7ff7493b2a2f+222591]
	(No symbol) [0x0x7ff7490d602c]
	GetHandleVerifier [0x0x7ff74977e408+4202328]
	BaseThreadInitThunk [0x0x7ffb39ca7374+20]
	RtlUserThreadStart [0x0x7ffb3a4bcc91+33]
